<a href="https://colab.research.google.com/github/sgr1118/Sentence-Embedding-Using-Korean-Corpora/blob/main/04_%EB%8B%A8%EC%96%B4_%EC%88%98%EC%A4%80_%EC%9E%84%EB%B2%A0%EB%94%A9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 단어 수준 임베딩
- 4장에서는 다양한 단어 수준 임베딩 모델을 알아본다. NPLM, Word2Vec, FastText 등은 예측 기반 모델이고, LSA, GloVe, Swivel 등은 행렬 분해 기반의 기법들이다. 가중 임베딩(Weighted Embedding)은 단어 임베딩 문장 수준으로 확장하는 방법이다.

## 4.1 NPLM
- NPLM(Neural Probabilistic Language Model)이다. NPLM은 NLP에서 임베딩 개념을 알리는데 일조한 모델로써 임베딩 역사에서 차지하는 역할이 작지 않다.

### 4.1.1 모델의 기본 구조
- NPLM은 2장 2.3에서 '단어가 어떤 순서로 쓰였는가'에서 설명한 통계 기반의 전통적인 언어 모델의 한계를 극복하고자 만들어졌다. 언어 모델의 단점은 아래와 같았다.

- 1. 학습 데이터에 존재하지 않는 n-gram이 포함된 문장이 나타날 확률 값을 0으로 부여한다. 백오프, 스무딩으로 문제를 보완할 수는 있지만 완전하지는 않다.
- 2. 문장의 장기 의존성을 포착하기 어렵다. 다시 말하면 n-gram 모델의 n을 5 이상으로 길게 설정할 수 없다. n이 커질수록 그 등장 확률이 0인 단어 시퀀스가 급격히 증가한다.
- 3. 단어/문장 간 유사도를 계산할 수 없다.

- NPLM은 이런 단점을 일부 극복했다는 의의가 있다. 그뿐만 아니라 NLPM 자체가 단어 임베딩 역할을 수행할 수 있다. NLPM의 아키텍터의 개괄적인 모습은 아래와 같다.

![](https://abhinavcreed13.github.io/blog/bengio-trigram-nplm-using-pytorch/bengio-model.png)
<center>NPLM 아키텍처</center>

### 4.1.2 NPLM의 학습
- NLPM은 단어 시퀀스가 주어질 때 다음 단어가 무엇인지 맞추는 과정에서 학습되다. NPLM은 직적까지 등장한 n-1개 단어들로 단어를 맞추는 n-gram 언어 모델이다. 예를 들어 '발','없는','말'이라는 단어가 주어질 때 그 다음 단어는 많은 것들이 올 수 있지만 우리가 가진 말뭉치가 '천리', '간다'라는 단어가 자주 등장한다고 해보자.

- NPLM은 아래 수식을 최대화하려고 한다. 이는 조건부 확률 P(천리|발, 없는, 말), P(간다|없는, 말, 천리)를 각각 높인다는 의미와 같다.

수식 4-1 NPLM의 출력
- $P(w_t|t_t-1, \cdots, w_t-n+1) = \frac{exp(y_{wt})}{\sum_i exp(y_i)}$
- $w_t$는 문장의 t번째 단어, 다시 말하면 w1 = 발, w2 = 없는, w3 = 말, 이렇게 세 개 단어를 입력받을 때, w4 = 천리 를 맞춘다는 이야기이다.

- NPLM 구조 말단의 출력은 $|V|$차원의 스코어 벡터 $y_{wt}$에 소프트맥스 함수를 적용한 $|V|$차원의 확률 벡터다. 이는 곧 위 수식에 정의된 조건부 확률과 같다.

- NPLM은 확률 벡토에서 가장 높은 요소의 인덱스에 해당하는 단어가 실제 정답 단어와 일치하도록 학습한다. 다음은 입력을 알아본다. 문장 내 t번째 단어(wt)에 대응하는 단어 벡터 $x_t$를 만드는 과정이다. 수식은 4-2와 같다.

- $|V| * m$크기를 갖는 커다란 행렬 C에서 $w_t$에 해당하는 벡터를 참조한 형태다. $|V|$는 어휘 집합 크기, m은 $x_t$의 차원 수다. C 행렬의 원소 값은 초기에 랜덤 설정한다.

수식 4-2 NLPM의 입력(1)
- $x_t = C(w_t)$

- 여기에서 참조란 다음 의미를 갖는다. 어휘 집합에 속한 단어가 5개 뿐이고 $w_t$가 이 가운데 네 번째하고 가정한다. 그러면 $C(w_t)$는 행렬 C와 $w_t$에 해당하는 원핫벡터를 내적한 것과 같다. 이는 C라는 행렬에서 $w_t$에 해당하는 행만 참조하는 것과 동일하다. 아래는 위 설명을 그림으로 나타낸 것이다.

그림 행렬 C 참조하기

- $C(w_t) = [0 0 0 1 0] * \begin{bmatrix}
11 & 18 & 25 \\
10 & 12 & 19 \\
4 & 6 & 13 \\
23 & 5 & 7 \\
17 & 24 & 1 \\
\end{bmatrix}$

- $\begin{bmatrix} = 23 & 5 & 7 \end{bmatrix}$

- 입력부터 알아보자. '없는', '말', '천리' 이렇게 세 단어가 주어질 때, '간다'라는 단어를 예측해야 하는 상황이다. 우선 세 개 각각의 단어의 인덱스 값을 확인한다. 수식 4-3처럼 세 개 단어에 해당하는 열 벡터를 C에서 참조한 뒤, 이 세 개 벡터를 묶어주면 NPLM의 입력 벡터 x가 된다. n은 고려 대상 n-gram의 개수다.

- 전체 문장이 '발 없는 말이 천리 간다'라면 입력 단어들(없는, 말이, 천리)과 예측 대상 단어(간다)를 포함해 모두 네 개이기에 n = 4, 예측 대상 단어가 문장에서 다섯 번째로 등장하는 t = 5이다.

수식 4-3 NPLM의 입력(2)
- $x = [x_{t-1}, x_{t-2}, \cdots, x_{t-n+1}]$

- NPLM 아키텍처 하단에서 상단으로 볼 때 파란색 선으로 그어진 레이어가 각각 입력층, 은닉층, 출력층이다. 스코어 벡터 $y_{wt}$를 계산하는 방식은 수식 4-4와 같다.

수식 4-4 NPLM 스코어 벡터 y 계산
- $y_{wt} = b + W_x + U\ tanh(d+H_x)$

- 마지막으로 $y_{wt}$에 소프트맥스 함수를 적용한 뒤 이를 정답 단어인 '간다'의 인덱스와 비교해 역전파하는 방법으로 학습이 이루어진다. NPLM 학습이 종료되면 행렬 C를 각 단어에 해당하는 m차원 임베딩으로 사용한다. 수식 4-5는 NPLM의 학습 파라미터 차원 수를 정리한 것이다.

수식 4-5 NPLM의 학습 파라미터

![](https://user-images.githubusercontent.com/53667002/68085609-04233600-fe86-11e9-99aa-32b4c894a374.png)
<center>NPLM의 학습 파라미터</center>

### 4.1.3 NPLM과 의미 정보
- NPLM은 단어의 의미를 담아내는 과정의 예시는 아래와 같다.

- 아래 문장은 벤지오 연구 팀이 예로 든 유사 문장이다.

The cat is walking in the bedroom

A dog was running in a room

The cat is running in a room

A dog is walking in a bedroom

The dog was walking in the room

- 우선 n-gram의 n = 4로 둔다고 해보자. NPLM은 직전 3단어를 가지고 그 다음 단어를 맞추는 과정에서 학습된다. 네 번째 단어가 'walking'인 문장은 아래와 같다. NPLM은 아래와 같은 문장을 입력받으면 'walking'이 출력되도록 학습한다.

- The cat is, A dog is, The dog was

- 'The', 'A',' cat', 'dog' ,'is' ,'was' 등에 해당하는 행렬 C의 행 벡터들은 'walking'을 맞추는 과정에서 학습 손실을 최소화하는 그래디언트를 받아 동일하게 업데이트된다. 결과적으로 'The', 'A',' cat', 'dog' ,'is' ,'was' 벡터가 벡터 공간에서 같은 방향으로 조금 움직인다는 이야기이다.

- NPLM은 그 자체로 언어 모델 역할을 수행할 수 있다. 예를 들어 학습 데이터에 없는 'The mouse is running in a room'이라는 문장의 등장 확률을 예측해야 할 때, 기존에 설명한 통계 기반 n-gram 모델은 학습 데이터에 한 번도 등장하지 않는 패턴에 0이라는 확률을 부여한다.

- 하지만 NPLM은 위 문장이 말뭉치에 없어도 문맥이 비슷한 다른 문장을 참고하여 확률을 부여한다.


## 4.2 Word2Vec
- Word2Vec은 구글이 발표한 기법으로 널리 쓰이는 단어 임베딩 기법이다. Skip-Gram과 CBOW라는 모델을 제안했고 나중에는 이 두 모델을 근간으로 하되 네거티브 샘플링 등 학습 최적화 기법을 제안하였다.

### 4.2.1 모델 기본 구조

![](https://camo.githubusercontent.com/ea6b76d9b0455a9ef046027fcb46bb0e07b97424d438d90ba5b3c621a4ad4568/68747470733a2f2f626c6f672e6b616b616f63646e2e6e65742f646e2f6d6435384e2f62747141435a73676677522f656f7345634768554e724f583233434c345342324b312f696d672e706e67)
<center>CBOW와 Skip-gram 모델의 기본 구조</center>

- CBOW는 주변에 있는 문맥 단어(content word)들를 가지고 타깃 단어 하나를 맞추는 과정에서 학습된다. Skip-gram 모델은 타깃 단어를 가지고 주변 문맥 단어가 무엇일지 에측하는 과정에서 학습된다.

- 모델의 기본 구조를 보면 CBOW의 {입력, 출력} = {문맥 단어 4개, 타깃 단어}인 반면 SKip-gram의 {입력, 출력} = {타깃 단어, 타깃 단어 t-2}, {타깃 단어, 타깃 단어 t-1}, {타깃 단어, 타깃 단어 t+1}, {타깃 단어, 타깃 단어 t+2} 이렇게 4개쌍이 된다.

- Skip-gram이 같은 말뭉치로도 더 많은 학습 데이터를 확보할 수 있어 임베딩 품질이 CBOW보다 좋은 경향이 있다. 이 파트에서는 Skip-gram을 중심으로 Word2Vec을 설명하도록 한다.

### 4.2.2 학습 데이터 구축
- Skip-gram 모델의 학습 데이터를 구축하는 과정을 다음과 같다. 예시 문장(개울가 에서 속옷 빨래 를 하는 남녀)은 한국어 위키백과에서 가져왔다.

표 Skip-gram 모델의 학습 데이터 구축

포지티브 샘플

|t|c|
|-|-|
|빨래|에서|
|빨래|속옷|
|빨래|를|
|빨래|하는|

네거티브 샘플

|t|c|
|-|-|
|빨래|책상|
|빨래|안녕|
|빨래|자동차|
|빨래|숫자|

|t|c|
|-|-|
|빨래|커피|
|빨래|떡|
|빨래|사과|
|빨래|노트북|

- 포지티브 샘플이란 타깃 단어(t)와 그 주변에 실제로 등장한 문맥 단어(c)가 쌍을 말한다. 네거티브 샘플은 타깃 단어와 그 주변에 등장하지 않는 단어(말뭉치에서 랜덤 추출) 쌍을 의미한다. 위 표는 윈도우는 2로 설정한 결과이다. 샘플을 생성할 때 타깃 단어 앞뒤 두 개씩만 고려했다는 것이다.

- Skip-gram 모델은 전체 말뭉치를 단어별로 슬라이딩하며 학습 데이터를 만든다. 결과적으로 Skip-gram은 같은 말뭉치를 두고도 엄청나게 많은 학습 데이터 쌍을 만들어낼 수 있다.

- Skip-gram의 학습 방식은 소프트맥스 때문에 계산량이 비교적 큰 편이다. 타깃 단어를 입력받아 문맥 단어를 직접 출력하는 모델을 학습하려면, 정답 문맥 단어가 나타날 확률을 높이고 나머지 단어들 확률은 그에 맞게 낮춰야 한다. 그런데 어휘 집합에 속한 단어수는 매우 많기 때문에 계산하려면 비효율적이다.

- 이런 비효율성을 개선하고자 타깃 단어와 문맥 단어 쌍이 주어질 때 해당 쌍이 포저티브인지 네거티브인지 이진 분류하는 과정에서 학습되고 이 방법을 네거티브 샘플링이라고 한다.(Skip-gram의 두 번째 논문 방식)

- 네거티브 방식으로 학습하면 1개의 포저티브와 k개의 네거티브만 계산(정확히는 매 스텝마다 차원 수가 2인 시그모이드를 k+1회 계산)하면 되기 때문에 모델을 1스텝에 전체 단어를 모두 계산(매 스텝마다 어휘 집합 크기만큼의 차원 수를 가진 소프트맥스를 1회 계산)해야 하는 기존 방법보다 계산량이 적어진다.

- 위 표에서 k = 2로 설정했기에, 모델은 하나의 스텝에서 1개의 포저티브 샘플과 2개의 네거티브 샘플, 즉 3개만 학습하면 된다. 논문에 따르면 작은 데이터는 k를 5 ~ 20, 큰 말뭉치에는 k 2 ~ 5로 설정하는 것이 성능이 좋았다고 한다.

- 네거티브 샘플 추출 확률은 아래 수식 4-6과 같다. 말뭉치에 자주 등장하지 않는 희귀한 단어가 네거티브 샘플로 더 잘 뽑힐 수 있도록 설계했다. $U(w_i)$는 해당 단어의 유니그램 확률(해당 단어 빈도/전체 단어 수)이다.

수식 4-6
- $P_{negative}(w_i) = \frac{U(w_i)^\frac{3}{4}}{\sum_{j=0}^n\ U(w_j)^\frac{3}{4}}$

- 예를 들어 말뭉치에 있는 단어가 '용', '미르'뿐이라면 그 구성 비율은 각각 0.99, 0.01이라고하자. 네거티브 샘플 확률을 구하면 아래와 같다.

수식 4-7 용과 미르가 네거티브 샘플로 뽑힐 확률
- $P(용) = \frac{0.99^0.75}{0.99^0.75 + 0.01^0.75} = 0.97$
- $P(미르) = \frac{0.01^0.75}{0.99^0.75 + 0.01^0.75} = 0.03$

- 원래라면 용은 0.99의 확률로 네거티브 샘플이 됐겠지만 0.97로 낮아졌다. 대신 미르는 0.01에서 0.03으로 높아졌다.

- 이와 별개로 자주 등장하는 단어는 학습에서 제외하는 서브샘플링이라는 기법도 적용했다. Skip-gram 모델은 말뭉치로부터 엄청나게 많은 학습 데이터 쌍을 만들어낼 수 있기 때문에 높은 빈도를 가진 단어의 경우 등장 횟수만큼 학습시키는 것이 비효율적이다. 서브샘플링 수식은 아래 수식 4-8과 같다. $f(w_i)$는 $w_i$의 빈도를 나타내며, $t$는 하이퍼파라미터이다. 논문에서는 $t = 10^-5$로 설정하였다.

수식 4-8 서브샘플링 확률
- $P_{subsampling}(w_i) = 1 - \sqrt{\frac{t}{f(w_i)}}$

- $f(w_i)$가 0.01로 나타나는 빈도 높은 단어(예를 들어 조사 은/는)는 위 식으로 계산할 시 0.9684로 계산된다. 즉, 100번의 학습 기회 가운데 96번 정도는 학습에서 제외한다. 반대로 등장 비율이 적어 $P_{subsampling}(w_i)$가 0에 가깝다면 해당 단어가 등장할 때마다 학습을 시키는 구조이다. 서브샘플링은 학습량을 효과적으로 줄여 계산량을 감소시키는 전략이다.

### 4.2.3 모델 학습
- Skip-gram 모델은 타깃 단어와 문맥 단어 쌍이 주어질 때, 그 쌍이 정말 포지티브인지 아닌지를 예측하는 과정에서 학습된다. 따라서 실제 포지티브 샘플이라면 아래 수식 4-9 조건부확률을 최대화해야 한다.

수식 4-9 t, c가 포지티브 샘플일 확률
- $P(+|t,c) = \frac{1}{1 + exp(-u_{t}v_{c})}$

- 위 수식에서 학습 파라미터는 $U$, $V$ 행렬 두 개뿐이다. 둘의 크기는 어휘 집합 크기(|V|) * 임베딩 차원 수(d)로 동일하다. $U$, $V$는 각각 타깃 단어와 문맥 단어에 대응한다. 위 수식을 최대화하려면 결국 분모를 줄여야 한다. 그럴러면 포지티브 샘플에 대응하는 단어 벡터인 $u_i$, $v_i$의 내적 값을 키워야 한다.

- 그런데 두 벡터의 내적은 코사인 유사도와 비례한다. 따라서 내적 값의 상향은 포지티브 샘플 t, c에 해당하는 단어 벡터 간 유사도를 높인다는 의미로 이해할 수 있다.

- Skip-gram 모델은 네거티브 샘플 단어 쌍에 관해 수식 4-10에 정의된 조건부 확률을 최대화해야한다. 다시 말하면 네거티브 샘플을 모델에 입력하면 모델은 이 데이터가 정말 네거티브 샘플이라고 맞춰야 한다는 것이다.

수식 4-10 t, c가 네거티브 샘플일 확률
- $P(-|t,c) = 1 - P(+|t,c) = \frac{exp(-u_{t}v_{c})}{1+exp(-u_{t}v_{c})}$

- 위 수식을 최대화하기 위해서는 분자 값을 높여야한다. 네거티브 샘플에 대응하는 단어 벡터인 $u_i$, $v_i$의 내적 값을 낮춰야 한다. 위에서 설명하였듯이 단어 벡터 간 코사인 유사도를 낮춘다는 의미로 이해할 수 있다.

- 수식 4-11은 Skip-gram 모델이 최대화해야하는 로그우도 함수다. 모델 파라미터인 $\theta$를 한 번 업데이트할 때 1개 쌍의 포지티브 샘플($t_p, c_p$)과 k개 네거티브 샘플($t_{ni}, c_{ni}$)이 학습된다는 것이다. 아래 수식을 최대화하는 과정에서 모델은 말뭉치의 분포 정보를 단어 임베딩에 함축시키는 것이다.

수식 4-11 Skip-gram 모델의 로그우드 함수
- $\mathcal{L} = logP(+|t_p, c_p) + \sum_{i=1}^k\ logP(-|t_{ni}, c_{ni})$

- 모델은 수식 4-11을 계산할 때 타깃 단어에 해당하는 단어 벡터 $u_i$는 행렬 U에서, 문맥 단어에 해당하는 단어 벡터 $v_i$는 행렬 V에서 참조한다. 모델 학습이 완료되면 U만 d차원의 단어 임베딩으로 쓸 수도 있고, $U + V^T$ 행렬을 임베딩으로 쓸 수도 있다. 혹은 $U, v^T$를 이어 붙여 2d 차원의 단어 임베딩으로 사용할 수도 있다.

### 4.2.4 듀토리얼
- Word2Vec의 Skip-gram 임베딩을 만들어보기로하고 임베딩 수행 대상 말뭉치는 한국어 위키백과, KorQuad, 네이버 영화 리뷰 말뭉치 3개 데이터 셋을 합치기로 한다.


In [10]:
# KoNLPy 설치
!curl -s https://raw.githubusercontent.com/teddylee777/machine-learning/master/99-Misc/01-Colab/mecab-colab.sh | bash

--2023-03-27 15:17:25--  https://www.dropbox.com/s/9xls0tgtf3edgns/mecab-0.996-ko-0.9.2.tar.gz?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.18, 2620:100:6021:18::a27d:4112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/dl/9xls0tgtf3edgns/mecab-0.996-ko-0.9.2.tar.gz [following]
--2023-03-27 15:17:26--  https://www.dropbox.com/s/dl/9xls0tgtf3edgns/mecab-0.996-ko-0.9.2.tar.gz
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc316bc0b35b131327d6fd0318ec.dl.dropboxusercontent.com/cd/0/get/B5AYJ7BbY3lcWWgWKiL7Yd1eZsZozKpc_PNxoaMCg1uF0z12knfWhQNbwwEQkJx9DvrC8rsUts9DSJ0an-40_5YAulLWbkQ86UGtBTTjnPa5gDgE4qcSy13VeB6hxQzn7JeNJIiUlSIEFbQDThoMhDXSJo7d1IDCqetTdjbIPxXyUpApIP72Xj52Bgfy49oQ0v4/file?dl=1# [following]
--2023-03-27 15:17:26--  https://uc316bc0b35b131327d6fd0318ec.dl.dropboxusercontent.com/cd/0/get/B5AYJ7BbY

In [14]:
cd '/content/drive/MyDrive/Book/Sentence_Embedding_Using_Korean_Corpora/data'

/content/drive/MyDrive/Book/Sentence_Embedding_Using_Korean_Corpora/data


In [ ]:
%ls text/AA

In [3]:
import os
import re

def list_wiki(dirname):
    filepaths = []
    filenames = os.listdir(dirname)
    for filename in filenames:
        filepath = os.path.join(dirname, filename)

        if os.path.isdir(filepath):
            # 재귀 함수
            filepaths.extend(list_wiki(filepath))
        else:
            find = re.findall(r"wiki_[0-9][0-9]", filepath)
            if 0 < len(find):
                filepaths.append(filepath)
    return sorted(filepaths)

In [4]:
filepaths = list_wiki('text')

In [5]:
len(filepaths)

947

In [6]:
with open("output_file.txt", "w") as outfile:
    for filename in filepaths:
        with open(filename) as infile:
            contents = infile.read()
            outfile.write(contents)

In [7]:
f = open('output_file.txt', encoding="utf8")

i = 0
while True:
    line = f.readline()
    if line != '\n':
        i = i+1
        print("%d번째 줄 :"%i + line)
    if i==10:
        break 
f.close()

1번째 줄 :<doc id="5" url="https://ko.wikipedia.org/wiki?curid=5" title="지미 카터">

2번째 줄 :지미 카터

3번째 줄 :제임스 얼 카터 주니어(, 1924년 10월 1일 ~ )는 민주당 출신 미국의 제39대 대통령(1977년 ~ 1981년) 이다.

4번째 줄 :생애.

5번째 줄 :어린 시절.

6번째 줄 :지미 카터는 조지아주 섬터 카운티 플레인스 마을에서 태어났다.

7번째 줄 :조지아 공과대학교를 졸업하였다. 그 후 해군에 들어가 전함·원자력·잠수함의 승무원으로 일하였다. 1953년 미국 해군 대위로 예편하였고 이후 땅콩·면화 등을 가꿔 많은 돈을 벌었다. 그의 별명이 "땅콩 농부" (Peanut Farmer)로 알려졌다.

8번째 줄 :정계 입문.

9번째 줄 :1962년 조지아주 상원 의원 선거에서 낙선하였으나, 그 선거가 부정선거 였음을 입증하게 되어 당선되고, 1966년 조지아 주지사 선거에 낙선하지만, 1970년 조지아 주지사 선거에서 당선 됬다. 대통령이 되기 전 조지아주 상원의원을 두번 연임했으며, 1971년부터 1975년까지 조지아 지사로 근무했다. 조지아 주지사로 지내면서, 미국에 사는 흑인 등용법을 내세웠다.

10번째 줄 :대통령 재임.



In [8]:
f = open('output_file.txt', encoding="utf8")
lines = f.read().splitlines()
print(len(lines))

10795052


In [11]:
# 한국어 위키백과 형태소 분석

from tqdm import tqdm
from konlpy.tag import Mecab 

mecab = Mecab()

result = []

for line in tqdm(lines):
  # 빈 문자열이 아닌 경우에만 수행
  if line:
    result.append(mecab.morphs(line))

100%|██████████| 10795052/10795052 [19:02<00:00, 9449.77it/s] 


In [12]:
with open('ko_wiki_mecab_train.txt', 'w', encoding='UTF-8') as f:
    for text in result:
        for name in text:
            f.write(name+'\n')

In [ ]:
# KorQuad 형태소 분석 (train)

from tqdm import tqdm
from konlpy.tag import Mecab 

f = open('/content/drive/MyDrive/Book/Sentence_Embedding_Using_Korean_Corpora/data/precessed_korquad_train.txt', encoding="utf8")
lines_kor = f.read().splitlines()
print(len(lines_kor))

70198


In [ ]:
type(lines_kor)

list

In [ ]:
mecab = Mecab()

result_kor = []

for line in tqdm(lines_kor):
  # 빈 문자열이 아닌 경우에만 수행
  if line:
    result_kor.append(mecab.morphs(line))

100%|██████████| 70198/70198 [00:28<00:00, 2492.55it/s]


In [ ]:
print(result_kor[1])

['바그너', '는', '괴테', '의', '파우스트', '를', '읽', '고', '무엇', '을', '쓰', '고자', '했', '는가', '?', '교향곡']


In [ ]:
result_kor[:3]

In [ ]:
with open('Korquad_mecab_train.txt', 'w', encoding='UTF-8') as f:
    for text in result_kor:
        for name in text:
            f.write(name+'\n')

In [ ]:
f = open('/content/Korquad_mecab_train.txt', encoding="utf8")
lines_kor = f.read().splitlines()
print(len(lines_kor))

3562826


In [ ]:
mv /content/Korquad_mecab_train.txt /content/drive/MyDrive/Book/Sentence_Embedding_Using_Korean_Corpora/data

In [ ]:
# KorQuad 형태소 분석 (dev)

from tqdm import tqdm
from konlpy.tag import Mecab 

f = open('/content/drive/MyDrive/Book/Sentence_Embedding_Using_Korean_Corpora/data/precessed_korquad_dev.txt', encoding="utf8")
lines_kor_dev = f.read().splitlines()
print(len(lines_kor_dev))

result_kor_dev = []

for line in tqdm(lines_kor_dev):
  # 빈 문자열이 아닌 경우에만 수행
  if line:
    result_kor_dev.append(mecab.morphs(line))

with open('korquad_mecab_dev.txt','w',encoding='UTF-8') as f:
    for text in result_kor_dev:
        for name in text:
            f.write(name+'\n')

6748


100%|██████████| 6748/6748 [00:02<00:00, 2928.48it/s]


In [ ]:
mv /content/korquad_mecab_dev.txt /content/drive/MyDrive/Book/Sentence_Embedding_Using_Korean_Corpora/data

In [16]:
# 네이버 영화 말뭉치 형태소 분석

from tqdm import tqdm
from konlpy.tag import Mecab 

f = open('/content/drive/MyDrive/Book/Sentence_Embedding_Using_Korean_Corpora/data/process_ratings.txt', encoding="utf8")
lines_kor_naver = f.read().splitlines()
print(len(lines_kor_naver))

result_kor_naver = []

for line in tqdm(lines_kor_naver):
  # 빈 문자열이 아닌 경우에만 수행
  if line:
    result_kor_naver.append(mecab.morphs(line))

with open('ratings_mecab.txt','w',encoding='UTF-8') as f:
    for name in result_kor_naver:
        for name in text:
            f.write(name+'\n')

199992


100%|██████████| 199992/199992 [00:23<00:00, 8367.22it/s]


In [21]:
# txt 파일 합치기

import glob, os

# 파일 불러오기
# 1. glob
# file_list = glob.glob('/', '*txt')
# 2. glob & os
# file_list = glob.glob(os.path.join('/', '*txt'))

file_list = glob.glob(os.path.join('/content/drive/MyDrive/Book/Sentence_Embedding_Using_Korean_Corpora/data/precessed_mecab', '*txt'))

with open('corpus_mecab.txt', 'w') as outfile:
    for filename in sorted(file_list):
        with open(filename) as file:
            for line in file:
                outfile.write(line)
# # 또는

# with open('/merge.txt', 'w') as outfile:
#     for filename in sorted(file_list):
#         with open(filename) as file:        
#             outfile.write(file.read())

In [ ]:
# 코드 4-3 Word2Vec Skip-gram 모델 학습

corpus_fname = "/content/drive/MyDrive/Book/Sentence_Embedding_Using_Korean_Corpora/data/precessed_mecab/corpus_mecab.txt"
model_fname = "/content/drive/MyDrive/Book/Sentence_Embedding_Using_Korean_Corpora/data/precessed_mecab/word2vec"

from gensim.models import Word2Vec
corpus = [sent.strip().split(" ") for sent in open(corpus_fname, 'r').readlines()]
model = Word2Vec(corpus, size = 100, workers = 4, sg = 1)
model.save(model_fname)

In [13]:
cd '/content/drive/MyDrive/Book/Sentence_Embedding_Using_Korean_Corpora/code/ratsgo-embedding-e7f7167/models'

/content/drive/MyDrive/Book/Sentence_Embedding_Using_Korean_Corpora/code/ratsgo-embedding-e7f7167/models


In [ ]:
# 코드 4-5 코사인 유사도 상위 단어 목록 체크 코드와 결과 예시

from models.word_eval import WordEmbeddingEvaluator
model = WordEmbeddingEvaluator('/content/drive/MyDrive/Book/Sentence_Embedding_Using_Korean_Corpora/data/precessed_mecab/word2vec',
                               method = 'word2vec', dim = 100, tokenizer_name = "mecab")
model.most_similar('희망', topn = 5)

## 4.3 FastText
- FastText는 메타에서 개발하여 배포한 단어 임베딩 기법이다. 각 단어를 문자 단위 n-gram으로 표현한다. 그 외 내용은 Word2Vec과 일치한다.

### 4.3.1 모델 기본 구조
- 만약 '시나브로'라는 단어의 문자 단위 n-gram(n=3)은 다음과 같다. <시나, 시나브, 나브로, 브로>, <시나브로>

- FastText는 '시나브로'라는 단어의 임베딩을 수식 4-12와 같이 5개의 문자 단위 n-gram 벡터의 합으로 표현한다. $G_i$는 타깃 단어 t에 속한 문자 단위 n-gram 집합을 말한다.

수식 4-12 FastText의 단어 벡터 표현
- $U_{시나브로} = Z_{시나} + Z_{시나브} + Z_{나브로} + Z_{브로>}+ Z_{시나브로}$
- $U_t = \sum_{g \in G_t} Z_{g}$

- FastText 모델 역시 네거티브 샘프링

## 3.4 요약
- 임베딩 학습용 말뭉치는 라인 하나가 문서면 좋다. 한국어 위키 백과, KorQuAD, 네이버 영화 말뭉치 등을 이같이 전처리했다.
- 지도 학습 기반의 형태소 분석 모델은 언어학 전문가들이 태깅한 형태소 분석 말뭉치로부터 학습된 기법이다. 이 모델들은 문자열이 주어질 때 사람이 정한 정답 패턴에 가깝게 토크나이즈한다. KoNLPy, Khaiii 등이 이 부류에 속한다.
- 비지도 학습 기반의 형태소 분석 모델은 데이터의 패턴을 모델 스스로 학습하게 함으로써 형태소를 나누는 기법이다. 데이터에 자주 등장하는 단어들을 형태소로 인식한다. soynlp, Google SentencePicec 등이 이 부류에 속한다.